# Combine CSV Files Into One Cleaned DataFrame

In [1]:
import pandas as pd
import datetime
from geopy.distance import vincenty

In [2]:
""" Loop though 3 CSV Files In January (Thursday-08, Friday-09, Saturday-10) and Add Together Into One File """

# Make main df with first file
df = pd.read_csv("siri.20121108.csv", header = None, low_memory = False)
df.columns = ["Timestamp", "LineId", "Direction", "JourneyPatternId", "TimeFrame", "VehicleJourneyId", "BusOperator", "Congestion", "Long", "Lat", "Delay", "BlockId", "VehicleId", "StopId", "AtStop"]

for i in range(9, 11):
    
    if i < 10:
        i = "0" + str(i)
    
    # Next file to add
    file = "siri.201211" + str(i) + ".csv"

    # Make DataFrame to add to main df
    df_to_add = pd.read_csv(file, header = None, low_memory = False)
    df_to_add.columns = ["Timestamp", "LineId", "Direction", "JourneyPatternId", "TimeFrame", "VehicleJourneyId", "BusOperator", "Congestion", "Long", "Lat", "Delay", "BlockId", "VehicleId", "StopId", "AtStop"]

    # Combine DataFrames
    df = df.append(df_to_add ,ignore_index = True)

In [3]:
df.shape

(4766421, 15)

In [4]:
df.head()

,Timestamp,LineId,Direction,JourneyPatternId,TimeFrame,VehicleJourneyId,BusOperator,Congestion,Long,Lat,Delay,BlockId,VehicleId,StopId,AtStop
0,1352332801000000,39,0,00390002,2012-11-07,3810,PO,0,-6.398326,53.393314,-296,39025,36049,4362,0
1,1352332801000000,27,0,00270001,2012-11-07,4958,RD,0,-6.393233,53.284412,-879,27014,33230,2622,0
2,1352332801000000,27,0,077A1001,2012-11-07,4993,RD,0,-6.246746,53.344143,-510,27023,33319,350,0
3,1352332801000000,76,0,00760001,2012-11-07,4553,CD,0,-6.361800,53.311016,-263,76001,33352,4341,1
4,1352332801000000,13,0,00131005,2012-11-07,4175,CD,0,-6.279008,53.342808,-156,13003,38063,1998,0


In [5]:
# Constant Column
del df["Direction"]

In [6]:
# Organise the Data
df = df.sort_values(['TimeFrame', 'VehicleId', 'VehicleJourneyId', 'Timestamp'], ascending=True)

# Clean up index
df = df.reset_index()
del df['index']

In [7]:
df.to_csv("raw_data.csv", index=False)

In [8]:
df.head()

,Timestamp,LineId,JourneyPatternId,TimeFrame,VehicleJourneyId,BusOperator,Congestion,Long,Lat,Delay,BlockId,VehicleId,StopId,AtStop
0,1352332811000000,16,00160001,2012-11-07,5591,SL,0,-6.267933,53.356716,-624,16013,24559,2975,1
1,1352332831000000,16,00160001,2012-11-07,5591,SL,0,-6.265700,53.356167,-624,16013,24559,2975,1
2,1352332852000000,16,00160001,2012-11-07,5591,SL,0,-6.265717,53.356152,-624,16013,24559,2975,1
3,1352332874000000,16,00160001,2012-11-07,5591,SL,0,-6.265666,53.356232,-624,16013,24559,2975,1
4,1352332892000000,16,00160001,2012-11-07,5591,SL,0,-6.265200,53.356033,-624,16013,24559,2975,1


## Clean Up Some Aspects of the Dataframe

In [9]:
""" Data from the 7th is not reliable, since some journey's continued from the previous day, we must remove it. """

df = df[df.TimeFrame != '2012-11-07']

In [10]:
""" Before each bus begins its journey it may not have a JourneyPatternId and records literal 'null' values instead
of actual data. These rows refer to instances where the bus is idle and hasn't moved anywhere, therefore they can 
confidently be deleted. """

# Filter out null values
df = df[df.JourneyPatternId != 'null']

In [11]:
# Organise the Data
df = df.sort_values(['TimeFrame', 'VehicleId', 'VehicleJourneyId', 'Timestamp'], ascending=True)

# Clean up index
df = df.reset_index()
del df['index']

In [12]:
# Check deletion worked
df.shape

(4052443, 14)

In [13]:
# Drop any duplicated rows
df = df.drop_duplicates()

In [33]:
# Organise the Data
df = df.sort_values(['TimeFrame', 'VehicleId', 'VehicleJourneyId', 'Timestamp'], ascending=True)

# Clean up index
df = df.reset_index()
del df['index']

In [15]:
df.shape

(4052443, 14)

In [16]:
df.to_csv("first_stage.csv", index=False)

In [17]:
df.head()

,Timestamp,LineId,JourneyPatternId,TimeFrame,VehicleJourneyId,BusOperator,Congestion,Long,Lat,Delay,BlockId,VehicleId,StopId,AtStop
0,1352356569000000,15,015B0002,2012-11-08,3276,RD,0,-6.233366,53.342232,0,15014,24549,395,1
1,1352356589000000,15,015B0002,2012-11-08,3276,RD,0,-6.233383,53.342251,0,15014,24549,395,1
2,1352356607000000,15,015B0002,2012-11-08,3276,RD,0,-6.233383,53.342251,0,15014,24549,395,1
3,1352356609000000,15,015B0002,2012-11-08,3276,RD,0,-6.233383,53.342216,0,15014,24549,395,1
4,1352356628000000,15,015B0002,2012-11-08,3276,RD,0,-6.233483,53.342033,0,15014,24549,395,1


In [34]:
""" If the GPS coordinates remain the same for the first few rows of data, delete them. """


for item, row in df.iterrows():
    
    # For the first iteration
    if item == 0:
        last_bus_id = row[4]
        last_lat = row[8]
        last_long = row[7]
        new_start = True
    
    # For every iteration
    current_bus_id = row[4]
    current_lat = row[8]
    current_long = row[7]
    
    # If it's a different vehicle journey id
    if last_bus_id != current_bus_id:
        last_bus_id = row[4]
        last_lat = row[8]
        last_long = row[7]
        new_start = True
        continue
  
    # If it's the same journey
    if new_start:
        if current_lat == last_lat:
            if current_long == last_long:
                df.set_value(item - 1, "VehicleJourneyId", 0)
                last_bus_id = row[4]
                last_lat = row[8]
                last_long = row[7]
                
    # If it's the same journey but it's moved
    if last_bus_id == current_bus_id:
        if current_lat != last_lat or current_long != last_long:
            new_start = False

In [35]:
# Filter Out Rows Flagged
df = df[df.VehicleJourneyId != 0]

In [36]:
# Organise the Data
df = df.sort_values(['TimeFrame', 'VehicleId', 'VehicleJourneyId', 'Timestamp'], ascending=True)

# Clean up index
df = df.reset_index()
del df['index']

In [37]:
df.to_csv("second_stage.csv", index=False)

In [95]:
# df = pd.read_csv("second_stage.csv", low_memory=False)

# Test to see if it worked

# test = df[df.VehicleJourneyId == 18]
# test[test.TimeFrame=='2012-11-08']

In [41]:
""" Now is the important part, there is sometimes noise in the null values previoulsy deleted, in order to only keep 
the relevent information for each journey we will now filter by removing rows that are more than 90 seconds apart before
the bus leaves the first terminal. Since the max space between two transmissions is 60sec, this should remove any outliers
which would mess up the distance/time columns to be added later. """


for item, row in df.iterrows():
    
    # For the first iteration
    if item == 0:
        last_bus_id = row[4]
        last_time = row[0]
        continue
    
    # For every iteration
    current_bus_id = row[4]
    current_time = row[0]
                
    # If it's a different vehicle journey id
    if last_bus_id != current_bus_id:
        last_bus_id = row[4]
        last_time = row[0]
        continue
        
    # If it's the same bus/journey
    if last_bus_id == current_bus_id:
        if current_time - last_time <= 90000000:
            last_bus_id = row[4]
            last_time = row[0]
            continue
  
    # If it's the same journey but an outlier has sneaked in...
    if current_time - last_time > 90000000:
        df.set_value(item - 1, "Congestion", 3)
        last_bus_id = row[4]
        last_time = row[0]

In [42]:
df = df[df.Congestion != 3]

In [44]:
# Organise the Data
df = df.sort_values(['TimeFrame', 'VehicleId', 'VehicleJourneyId', 'Timestamp'], ascending=True)

# Clean up index
df = df.reset_index()
del df['index']

In [45]:
df.head()

,Timestamp,LineId,JourneyPatternId,TimeFrame,VehicleJourneyId,BusOperator,Congestion,Long,Lat,Delay,BlockId,VehicleId,StopId,AtStop
0,1.352357e+15,15,015B0002,2012-11-08,3276.0,RD,0.0,-6.233366,53.342232,0.0,15014.0,24549.0,395,1.0
1,1.352357e+15,15,015B0002,2012-11-08,3276.0,RD,0.0,-6.233383,53.342251,0.0,15014.0,24549.0,395,1.0
2,1.352357e+15,15,015B0002,2012-11-08,3276.0,RD,0.0,-6.233383,53.342251,0.0,15014.0,24549.0,395,1.0
3,1.352357e+15,15,015B0002,2012-11-08,3276.0,RD,0.0,-6.233383,53.342216,0.0,15014.0,24549.0,395,1.0
4,1.352357e+15,15,015B0002,2012-11-08,3276.0,RD,0.0,-6.233483,53.342033,0.0,15014.0,24549.0,395,1.0


In [46]:
df.shape

(3951116, 14)

In [47]:
df.to_csv("third_stage.csv", index=False)

In [48]:
""" When there is a stream of AtStop == 1 at the start of a journey, these should all be deleted
except for the last one. """


for item, row in df.iterrows():
    
    # For the first iteration
    if item == 0:
        last_bus_id = row[4]
        last_at_stop = row[13]
        new_start = True
    
    # For every iteration
    current_bus_id = row[4]
    current_at_stop = row[13]
    
    # If it's left the terminal or didn't start there
    if current_at_stop == 0:
        new_start = False
    
    # If it's staying in the terminal
    if current_at_stop == 1 and last_at_stop == 1:
        if new_start:
            df.set_value(item - 1, "VehicleJourneyId", 0)
            
    # If it's a new bus
    if current_bus_id != last_bus_id:
        new_start = True
        last_bus_id = row[4]
        last_at_stop = row[13]

In [49]:
# Filter Out Rows
df = df[df.VehicleJourneyId != 0]

In [50]:
# Check
df.shape

(3908850, 14)

In [51]:
# Organise the Data
df = df.sort_values(['TimeFrame', 'VehicleId', 'VehicleJourneyId', 'Timestamp'], ascending=True)

# Clean up index
df = df.reset_index()
del df['index']

In [52]:
df.to_csv("fourth_stage.csv", index=False)

In [53]:
""" Some journeys do not register initial stop, we change atStop == 2 when this happens
for the first TimeStamp in each VehicleJourneyId. We choose 2 instead of 1 to identify them later if necessary. """

for item, row in df.iterrows():
    
    if item == 0:
        lastid = row[4]
        if row[13] ==  0:
            df.set_value(item, 'AtStop', 2)
        continue
        
    currentid = row[4]
    
    if currentid != lastid:
        lastid = currentid
        if row[13] ==  0:
            df.set_value(item, 'AtStop', 2)

In [54]:
df.to_csv("fifth_stage.csv", index=False)

## Everything Works Up To Here

In [55]:
""" Many VehicleJourneyId's have only one row, therefore they tell us nothing useful. These rows can confidently be deleted. """

""" Note: I'm not sure this works the way I'd hoped? """

count = 0

for item, row in df.iterrows():
    
    if item == 0:
        last = row[4]
        count += 1
        continue
            
    current = row[4]
    
    if current == last:
        last = row[4]
        count += 1
        continue
        
    if current != last:
        if count == 1:
            # Placeholder
            df.set_value(item - 1, "VehicleJourneyId", 0)
        count = 1
        last = row[4]

In [58]:
# Filter Out Rows
df = df[df.VehicleJourneyId != 0]

In [59]:
# Organise the Data
df = df.sort_values(['TimeFrame', 'VehicleId', 'VehicleJourneyId', 'Timestamp'], ascending=True)

# Clean up index
df = df.reset_index()
del df['index']

In [60]:
df.shape

(3900231, 14)

In [61]:
# Check that the last entry isn't a unique Vehicle Journey ID because the previous loop will have missed it
df.head()

,Timestamp,LineId,JourneyPatternId,TimeFrame,VehicleJourneyId,BusOperator,Congestion,Long,Lat,Delay,BlockId,VehicleId,StopId,AtStop
0,1.352357e+15,15,015B0002,2012-11-08,3276.0,RD,0.0,-6.233483,53.341969,0.0,15014.0,24549.0,395,1.0
1,1.352357e+15,15,015B0002,2012-11-08,3276.0,RD,0.0,-6.233750,53.341816,0.0,15014.0,24549.0,396,0.0
2,1.352357e+15,15,015B0002,2012-11-08,3276.0,RD,0.0,-6.233750,53.341816,0.0,15014.0,24549.0,396,0.0
3,1.352357e+15,15,015B0002,2012-11-08,3276.0,RD,0.0,-6.235717,53.342083,0.0,15014.0,24549.0,396,0.0
4,1.352357e+15,15,015B0002,2012-11-08,3276.0,RD,0.0,-6.235717,53.342083,0.0,15014.0,24549.0,396,0.0


In [62]:
df.to_csv("sixth_stage.csv", index=False)

In [91]:
""" Remove every vehicle journey id which is less than 4 rows long. This is likely just noise in the data. It will also
help to get rid of vehicles which broadcasted the wrong vehicle journey id during their idle NULL patterns. """

x = df.groupby(["TimeFrame", "VehicleId", "VehicleJourneyId"])

x = x.filter(lambda x: len(x) >= 4)

df = pd.DataFrame(x)

In [92]:
df.head()

,Timestamp,LineId,JourneyPatternId,TimeFrame,VehicleJourneyId,BusOperator,Congestion,Long,Lat,Delay,BlockId,VehicleId,StopId,AtStop
0,1.352357e+15,15,015B0002,2012-11-08,3276.0,RD,0.0,-6.233483,53.341969,0.0,15014.0,24549.0,395,1.0
1,1.352357e+15,15,015B0002,2012-11-08,3276.0,RD,0.0,-6.233750,53.341816,0.0,15014.0,24549.0,396,0.0
2,1.352357e+15,15,015B0002,2012-11-08,3276.0,RD,0.0,-6.233750,53.341816,0.0,15014.0,24549.0,396,0.0
3,1.352357e+15,15,015B0002,2012-11-08,3276.0,RD,0.0,-6.235717,53.342083,0.0,15014.0,24549.0,396,0.0
4,1.352357e+15,15,015B0002,2012-11-08,3276.0,RD,0.0,-6.235717,53.342083,0.0,15014.0,24549.0,396,0.0


In [96]:
# Organise the Data
df = df.sort_values(['TimeFrame', 'VehicleId', 'VehicleJourneyId', 'Timestamp'], ascending=True)

# Clean up index
df = df.reset_index()
del df['index']

In [97]:
df.shape

(3898305, 14)

In [93]:
df.to_csv("seventh_stage.csv", index=False)

## Add Distance Feature

Before removing all information in the CSV where the feature AtStop == 0 we must first measure the distance on each route. This requires that we use all rows of data. 

In order to cut down the time of this loop (which will be costly) first delete all rows which have a literal 'null' value for JourneyPatternId, clean up the noise before each bus journey and finally remove all unique values for VehicleJourneyId. This will significantly reduce the size of the dataframe and the subsequent loop below.

In [98]:
def get_distance(lat1, long1, lat2, long2):
    """ Get distance between two geo coordinates """
    
    stop1 = (lat1, long1)
    stop2 = (lat2, long2)
    
    return vincenty(stop1, stop2).meters

In [99]:
# List to hold feature
distance = list()

for item, row in df.iterrows():
    
    # Set up values on first iteration
    if item == 0:
        last_lat = row[8]
        last_long = row[7]
        last_id = row[4]
        last_distance = 0
        
        distance.append(0)
        continue
    
    current_id = row[4]
    current_lat = row[8]
    current_long = row[7]
    current_distance = get_distance(current_lat, current_long, last_lat, last_long)
    
    # If it's a new Journey ID
    if current_id != last_id:
        last_id = row[4]
        last_lat = row[8]
        last_long = row[7]
        last_distance = 0
        
        distance.append(0)
        continue
     
    # If it's not a new Journey ID
    current_distance = get_distance(current_lat, current_long, last_lat, last_long)
    distance.append(last_distance + current_distance)
    
    last_distance = distance[-1]
    last_lat = row[8]
    last_long = row[7]
    last_id = row[4]

In [100]:
df["Distance"] = distance

In [101]:
df.head()

,Timestamp,LineId,JourneyPatternId,TimeFrame,VehicleJourneyId,BusOperator,Congestion,Long,Lat,Delay,BlockId,VehicleId,StopId,AtStop,Distance
0,1.352357e+15,15,015B0002,2012-11-08,3276.0,RD,0.0,-6.233483,53.341969,0.0,15014.0,24549.0,395,1.0,0.000000
1,1.352357e+15,15,015B0002,2012-11-08,3276.0,RD,0.0,-6.233750,53.341816,0.0,15014.0,24549.0,396,0.0,24.621231
2,1.352357e+15,15,015B0002,2012-11-08,3276.0,RD,0.0,-6.233750,53.341816,0.0,15014.0,24549.0,396,0.0,24.621231
3,1.352357e+15,15,015B0002,2012-11-08,3276.0,RD,0.0,-6.235717,53.342083,0.0,15014.0,24549.0,396,0.0,158.962019
4,1.352357e+15,15,015B0002,2012-11-08,3276.0,RD,0.0,-6.235717,53.342083,0.0,15014.0,24549.0,396,0.0,158.962019


In [103]:
df.to_csv("eight_stage.csv", index=False)

## Change Unix Timestamp to Human Readable Format

In [105]:
# Must be int for division next
df['Timestamp'] = df['Timestamp'].apply(int)

In [107]:
# Change date in bus file to be in standard format instead of unix milliseconds (1:45 mins)
for index, row in df.iterrows():

    x = row[0]/1000000
    df.set_value(index, 'Timestamp', x) 

In [108]:
# For formatting later
df['Timestamp'] = df['Timestamp'].apply(str)

In [109]:
# Change format of date 
for index, row in df.iterrows():

    ts = int(row[0])
    x = datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')
    df.set_value(index, 'Timestamp', str(x))

In [110]:
# Change Features To Correct Types
df['Timestamp'] =  pd.to_datetime(df['Timestamp'], format='%Y-%m-%d %H:%M:%S')
df["LineId"] = df["LineId"].astype("category")
df["JourneyPatternId"] = df["JourneyPatternId"].astype("category")
df["TimeFrame"] = df["TimeFrame"].astype("category")
df["VehicleJourneyId"] = df["VehicleJourneyId"].astype("category")
df["Congestion"] = df["Congestion"].astype("category")
df["BlockId"] = df["BlockId"].astype("category")
df["StopId"] = df["StopId"].astype("category")
df["BusOperator"] = df["BusOperator"].astype("category")
df["VehicleId"] = df["VehicleId"].astype("category")

In [111]:
df.dtypes

Timestamp           datetime64[ns]
LineId                    category
JourneyPatternId          category
TimeFrame                 category
VehicleJourneyId          category
BusOperator               category
Congestion                category
Long                       float64
Lat                        float64
Delay                      float64
BlockId                   category
VehicleId                 category
StopId                    category
AtStop                     float64
Distance                   float64
dtype: object

In [112]:
# Organise the Data
df = df.sort_values(['TimeFrame', 'VehicleId', 'VehicleJourneyId', 'Timestamp'], ascending=True)

# Clean up index
df = df.reset_index()
del df['index']

In [113]:
df.shape

(3898305, 15)

In [114]:
df.to_csv("ninth_stage.csv", index=False)

In [158]:
df = pd.read_csv("ninth_stage.csv", low_memory=False)

## Remove AtStop == 0 & Drop Duplicates

In [159]:
# Now it is safe to filter the dataframe down to when buses are at stops
# So stop will be values of 1 and 2 (we added two as a fake stop for some routes)
df = df[df.AtStop != 0]

In [160]:
# Organise the Data
df = df.sort_values(['TimeFrame', 'VehicleId', 'VehicleJourneyId', 'Timestamp'], ascending=True)

# Clean up index
df = df.reset_index()
del df['index']

In [162]:
df.shape

(870462, 15)

In [163]:
""" If the notebook has been followed properly then this cell may be run without loss of valuable data 

It will delete all duplicates AFTER the first example of each is found. So when the bus arrives at the stop, all subsequent rows
at that stop will be deleted. """

df = df.drop_duplicates(["TimeFrame", 'VehicleId', "VehicleJourneyId", "StopId"])

In [164]:
# Organise the Data
df = df.sort_values(['TimeFrame', 'VehicleId', 'VehicleJourneyId', 'Timestamp'], ascending=True)

# Clean up index
df = df.reset_index()
del df['index']

In [165]:
# Check size of dataframe
df.shape

(393268, 15)

In [166]:
df.head()

,Timestamp,LineId,JourneyPatternId,TimeFrame,VehicleJourneyId,BusOperator,Congestion,Long,Lat,Delay,BlockId,VehicleId,StopId,AtStop,Distance
0,2012-11-08 06:37:28,15,015B0002,2012-11-08,3276.0,RD,0.0,-6.233483,53.341969,0.0,15014.0,24549.0,395,1.0,0.000000
1,2012-11-08 06:48:23,15,015B0002,2012-11-08,3276.0,RD,0.0,-6.256696,53.342678,69.0,15014.0,24549.0,403,1.0,2706.310938
2,2012-11-08 06:57:04,15,015B0002,2012-11-08,3276.0,RD,0.0,-6.266577,53.321621,58.0,15014.0,24549.0,1077,1.0,5525.509641
3,2012-11-08 07:16:52,15,015B0002,2012-11-08,3276.0,RD,0.0,-6.327800,53.270649,-309.0,15014.0,24549.0,6280,1.0,15187.807339
4,2012-11-08 07:50:33,15,015B1001,2012-11-08,3277.0,RD,0.0,-6.326270,53.271095,28.0,15014.0,24549.0,6285,1.0,0.000000


In [168]:
df.dtypes

Timestamp            object
LineId               object
JourneyPatternId     object
TimeFrame            object
VehicleJourneyId    float64
BusOperator          object
Congestion          float64
Long                float64
Lat                 float64
Delay               float64
BlockId             float64
VehicleId           float64
StopId               object
AtStop              float64
Distance            float64
dtype: object

In [169]:
df['Timestamp'] =  pd.to_datetime(df['Timestamp'], format='%Y-%m-%d %H:%M:%S')

## Add Time Taken Feature

In [170]:
# List to hold features
travel_time = list()

for item, row in df.iterrows():
    
    # Set up values on first iteration
    if item == 0:
        last_id = row[4]
        start_time = row[0]
        travel_time.append(0.0)
        continue
    
    current_time = row[0]
    current_id = row[4]
    
    # If it's a new Journey ID
    if current_id != last_id:
        last_id = row[4]        
        start_time = row[0]        
        travel_time.append(0.0)
        continue
        
    travel_time.append(abs((current_time - start_time).total_seconds()))    
    last_time = row[0]
    last_id = row[4]

In [171]:
df["TravelTime"] = travel_time

In [172]:
df.head()

,Timestamp,LineId,JourneyPatternId,TimeFrame,VehicleJourneyId,BusOperator,Congestion,Long,Lat,Delay,BlockId,VehicleId,StopId,AtStop,Distance,TravelTime
0,2012-11-08 06:37:28,15,015B0002,2012-11-08,3276.0,RD,0.0,-6.233483,53.341969,0.0,15014.0,24549.0,395,1.0,0.000000,0.0
1,2012-11-08 06:48:23,15,015B0002,2012-11-08,3276.0,RD,0.0,-6.256696,53.342678,69.0,15014.0,24549.0,403,1.0,2706.310938,655.0
2,2012-11-08 06:57:04,15,015B0002,2012-11-08,3276.0,RD,0.0,-6.266577,53.321621,58.0,15014.0,24549.0,1077,1.0,5525.509641,1176.0
3,2012-11-08 07:16:52,15,015B0002,2012-11-08,3276.0,RD,0.0,-6.327800,53.270649,-309.0,15014.0,24549.0,6280,1.0,15187.807339,2364.0
4,2012-11-08 07:50:33,15,015B1001,2012-11-08,3277.0,RD,0.0,-6.326270,53.271095,28.0,15014.0,24549.0,6285,1.0,0.000000,0.0


In [173]:
df.to_csv("tenth_stage.csv", index=False)

## Add Weekday Feature

In [174]:
df['Weekday'] = df['Timestamp'].dt.dayofweek

In [175]:
# Convert to correct datatype
df["Weekday"] = df["Weekday"].astype("category")

## Add Time Category

In [176]:
# Convert to temp datatype
df["Timestamp"] = df["Timestamp"].astype(str)

In [177]:
time_cat = list()

for item, row in df.iterrows():
    
    s = row[0][-8:]
    temp = s.split(":")

    if int(temp[1]) < 30:
        mins = "00"
    else:
        mins = "30"

    ans = temp[0] + ":" + mins
    
    time_cat.append(ans)

In [178]:
df["TimeCategory"] = time_cat

In [179]:
df.head()

,Timestamp,LineId,JourneyPatternId,TimeFrame,VehicleJourneyId,BusOperator,Congestion,Long,Lat,Delay,BlockId,VehicleId,StopId,AtStop,Distance,TravelTime,Weekday,TimeCategory
0,2012-11-08 06:37:28,15,015B0002,2012-11-08,3276.0,RD,0.0,-6.233483,53.341969,0.0,15014.0,24549.0,395,1.0,0.000000,0.0,3,06:30
1,2012-11-08 06:48:23,15,015B0002,2012-11-08,3276.0,RD,0.0,-6.256696,53.342678,69.0,15014.0,24549.0,403,1.0,2706.310938,655.0,3,06:30
2,2012-11-08 06:57:04,15,015B0002,2012-11-08,3276.0,RD,0.0,-6.266577,53.321621,58.0,15014.0,24549.0,1077,1.0,5525.509641,1176.0,3,06:30
3,2012-11-08 07:16:52,15,015B0002,2012-11-08,3276.0,RD,0.0,-6.327800,53.270649,-309.0,15014.0,24549.0,6280,1.0,15187.807339,2364.0,3,07:00
4,2012-11-08 07:50:33,15,015B1001,2012-11-08,3277.0,RD,0.0,-6.326270,53.271095,28.0,15014.0,24549.0,6285,1.0,0.000000,0.0,3,07:30


## Make Final Changes To Datatypes

In [180]:
df.dtypes

Timestamp             object
LineId                object
JourneyPatternId      object
TimeFrame             object
VehicleJourneyId     float64
BusOperator           object
Congestion           float64
Long                 float64
Lat                  float64
Delay                float64
BlockId              float64
VehicleId            float64
StopId                object
AtStop               float64
Distance             float64
TravelTime           float64
Weekday             category
TimeCategory          object
dtype: object

In [181]:
df["VehicleJourneyId"] = df["VehicleJourneyId"].astype(int)
df["Congestion"] = df["Congestion"].astype(int)
df["BlockId"] = df["BlockId"].astype(int)
df["VehicleId"] = df["VehicleId"].astype(int)
df["Delay"] = df["Delay"].astype(int)
df["TravelTime"] = df["TravelTime"].astype(int)

In [182]:
df["VehicleJourneyId"] = df["VehicleJourneyId"].astype('category')
df["Congestion"] = df["Congestion"].astype('category')
df["BlockId"] = df["BlockId"].astype('category')
df["VehicleId"] = df["VehicleId"].astype('category')
df["TimeCategory"] = df["TimeCategory"].astype('category')
df['Timestamp'] =  pd.to_datetime(df['Timestamp'], format='%Y-%m-%d %H:%M:%S')

In [183]:
df.dtypes

Timestamp           datetime64[ns]
LineId                      object
JourneyPatternId            object
TimeFrame                   object
VehicleJourneyId          category
BusOperator                 object
Congestion                category
Long                       float64
Lat                        float64
Delay                        int32
BlockId                   category
VehicleId                 category
StopId                      object
AtStop                     float64
Distance                   float64
TravelTime                   int32
Weekday                   category
TimeCategory              category
dtype: object

In [184]:
df.head()

,Timestamp,LineId,JourneyPatternId,TimeFrame,VehicleJourneyId,BusOperator,Congestion,Long,Lat,Delay,BlockId,VehicleId,StopId,AtStop,Distance,TravelTime,Weekday,TimeCategory
0,2012-11-08 06:37:28,15,015B0002,2012-11-08,3276,RD,0,-6.233483,53.341969,0,15014,24549,395,1.0,0.000000,0,3,06:30
1,2012-11-08 06:48:23,15,015B0002,2012-11-08,3276,RD,0,-6.256696,53.342678,69,15014,24549,403,1.0,2706.310938,655,3,06:30
2,2012-11-08 06:57:04,15,015B0002,2012-11-08,3276,RD,0,-6.266577,53.321621,58,15014,24549,1077,1.0,5525.509641,1176,3,06:30
3,2012-11-08 07:16:52,15,015B0002,2012-11-08,3276,RD,0,-6.327800,53.270649,-309,15014,24549,6280,1.0,15187.807339,2364,3,07:00
4,2012-11-08 07:50:33,15,015B1001,2012-11-08,3277,RD,0,-6.326270,53.271095,28,15014,24549,6285,1.0,0.000000,0,3,07:30


In [185]:
df.to_csv("eleventh_stage.csv", index=False)

## Fix Last Two Issues

### Summary Of Issues:
* After having filtered the information to when buses are at stops only (or our 'made up' stop 2), there are again some journeys which are just a single row long, these are useless and should be removed.
* 6.7% of the journeys also have an incorrect JourneyPatternId, these should ideally be changed to the correct label

After this the data clean should be correct enough to start modelling

### Remove journey's which are only 1 row of data in length (noise)

In [188]:
# For reference
df.shape

(393268, 18)

In [ ]:
392687

In [187]:
# Demonstrate Issue
test = df[df.TimeFrame == "2012-11-08"]
test.groupby(["VehicleId", "VehicleJourneyId"]).size()

VehicleId  VehicleJourneyId
24549      3276                 4
           3277                27
           4935                 3
           4936                21
24559      154                 20
           155                 23
           156                  3
           157                 15
           158                 22
           159                 18
           160                  8
           161                 12
           162                  4
           3938                 2
           5625                26
           5626                39
24560      5646                35
           5647                36
           6167                28
           6168                19
24574      1853                 1
           1854                 1
           1855                 1
           1856                 1
           1859                 1
           1860                 1
           1861                 2
           1865                 1
           1866     

In [189]:
""" Remove every VehicleJourneyId which is 1 row in length. We may want to widen this filter later, but for now these
are definitely just noise in the data. """

x = df.groupby(["TimeFrame", "VehicleId", "VehicleJourneyId"])

x = x.filter(lambda x: len(x) > 1)

df = pd.DataFrame(x)

In [190]:
# Check to see if it worked
df.shape

(392687, 18)

Deleted 581 Journeys/Rows (All just noise in the data)

In [206]:
df.to_csv("twelth_stage.csv", index=False)

### Change the JourneyPatternId's to the correct values

In [207]:
# Demonstrate Issue
x = df.groupby(["TimeFrame", "VehicleId", "VehicleJourneyId"])["JourneyPatternId"].unique()
x

TimeFrame   VehicleId  VehicleJourneyId
2012-11-08  24549      3276                          [015B0002]
                       3277                          [015B1001]
                       4935                          [077A0001]
                       4936                          [077A1001]
            24559      154                           [041C1001]
                       155                           [00410001]
                       156                           [041C1001]
                       157                           [041C0001]
                       158                           [041C1001]
                       159                           [041C0001]
                       160                           [00411001]
                       161                           [041C0001]
                       162                           [00411001]
                       3938                          [00411001]
                       5625                          [00160003]


In [208]:
# Measure the issue
count = 0
for item in x:
    if len(item) == 2:
        count += 1    
count

1431

So 1431 VehicleJourneyId's have a mixed JourneyPatternId. If possible we must change them to the correct ones.

## Daniele Add The Weather Information Here???

## Make CSV

In [122]:
df.to_csv('November_8_to_10.csv', index=False)